In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
import boto3
import pickle
import io
from urllib.parse import urlparse

def load_pickle_from_s3(s3_uri):
    # S3 URIを分解
    parsed = urlparse(s3_uri)
    bucket = parsed.netloc
    key = parsed.path.lstrip("/")

    # S3からバイナリデータを取得
    s3 = boto3.client("s3")
    response = s3.get_object(Bucket=bucket, Key=key)
    binary_data = response['Body'].read()

    print("✅ 読み込み完了:", s3_uri)

    # pickleとして読み込む
    return pickle.load(io.BytesIO(binary_data))

# 使用例
train_X = load_pickle_from_s3("s3://choitore-ai/Processed/train_X.pkl")
train_Y = load_pickle_from_s3("s3://choitore-ai/Processed/train_Y.pkl")
test_X = load_pickle_from_s3("s3://choitore-ai/Processed/test_X.pkl")




✅ 読み込み完了: s3://choitore-ai/Processed/train_X.pkl
✅ 読み込み完了: s3://choitore-ai/Processed/train_Y.pkl
✅ 読み込み完了: s3://choitore-ai/Processed/test_X.pkl


In [3]:
# CrossValidation用のデータ分割設定処理
from sklearn.model_selection import KFold
folds=3
kf = KFold(n_splits=folds)

In [4]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# parameta settings
lgbm_params = {
    "Objective": "binary",
    "random_seed": 1234
}
stopping_rounds=20
verbose=True
callbacks = [lgb.early_stopping(stopping_rounds=stopping_rounds, verbose=verbose)]
num_boost_round=100

# model格納用リスト
models = []
acc = 0
pre = 0
rec = 0
f = 0
for train_index, val_index in kf.split(train_X):
    X_train = train_X.iloc[train_index]
    X_valid = train_X.iloc[val_index]
    Y_train = train_Y.iloc[train_index]
    Y_valid = train_Y.iloc[val_index]

    lgb_train = lgb.Dataset(X_train, Y_train)
    lgb_eval = lgb.Dataset(X_valid, Y_valid, reference=lgb_train)

    model_lgb = lgb.train(lgbm_params,
                          lgb_train,
                          valid_sets=lgb_eval,
                          num_boost_round=num_boost_round,
                          callbacks=callbacks
                         )
    y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
    
    acc += accuracy_score(Y_valid, np.round(y_pred))
    pre += precision_score(Y_valid, np.round(y_pred), average='weighted')
    rec += recall_score(Y_valid, np.round(y_pred), average='weighted')
    f += f1_score(Y_valid, np.round(y_pred), average='weighted')
    
    print(accuracy_score(Y_valid, np.round(y_pred)))

[LightGBM] [Warning] Unknown parameter: Objective
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: Objective
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000357 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 559
[LightGBM] [Info] Number of data points in the train set: 3598, number of used features: 23
[LightGBM] [Warning] Unknown parameter: Objective
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.525848
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[27]	valid_0's l2: 0.18022
0.7133333333333334
[LightGBM] [Warning] Unknown parameter: Objective
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Wa

In [10]:
# 最終的なモデル作成
lgb_train = lgb.Dataset(train_X, train_Y)
model_lgb = lgb.train(lgbm_params,
                      lgb_train,
                     )

[LightGBM] [Warning] Unknown parameter: Objective
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: Objective
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000674 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 559
[LightGBM] [Info] Number of data points in the train set: 5398, number of used features: 23
[LightGBM] [Info] Start training from score 0.507225


In [11]:
# Replace this with the ARN of the Tracking Server you just created
arn = 'arn:aws:sagemaker:ap-northeast-1:323189574151:mlflow-tracking-server/Choitore'
mlflow.set_tracking_uri(arn)
mlflow.set_experiment("titanic")

with mlflow.start_run():
    # Log the evaluation metrics
    mlflow.log_metric("accuracy", acc/folds)
    mlflow.log_metric("precision", pre/folds)
    mlflow.log_metric("recall", rec/folds)
    mlflow.log_metric("f1_score", f/folds)

    mlflow.log_params(
        {
            "Objective": lgbm_params["Objective"],
            "random_seed": lgbm_params["random_seed"],
            "stopping_rounds": stopping_rounds,
            "verbose": verbose,
            "num_boost_round": num_boost_round,
        }
    )

    # Log the model
    mlflow.sklearn.log_model(
            sk_model=model_lgb,
            artifact_path="lgbmodel",
            input_example=train_X,
            registered_model_name="LGBModel",
    )

/opt/conda/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'LGBModel' already exists. Creating a new version of this model...
2025/08/01 09:38:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LGBModel, version 2

🏃 View run dashing-bass-911 at: https://ap-northeast-1.experiments.sagemaker.aws/#/experiments/34/runs/7efdd9b64992472b80b02cf182eab2ca
🧪 View experiment at: https://ap-northeast-1.experiments.sagemaker.aws/#/experiments/34


Created version '2' of model 'LGBModel'.


In [ ]:
preds = []
for model in models:
    pred = model.predict(test_X)
    preds.append(pred)

In [ ]:
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis=0)

In [ ]:
preds_int = (preds_mean > 0.5).astype(int)

In [ ]:
# 提出データサンプル取得
submission_df = pd.read_csv("submission_sample.csv")
submission_df

In [ ]:
submission_df["Transported"]= preds_int.astype(bool)
submission_df

In [ ]:
submission_df.to_csv("s3://choitore-ai/submissions/submission.csv")